# Simple deep multi-layer perceptron Sentinel 2 for the classification of the urban and agriculture areas

This notebook walks you through a simple example of using Earth Engine and Keras.

Specifically, we will train a neural network to recognize land, water, urbar, and cropland pixels in a Sentinel 2 scene ([gee](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2)). 
For this simple example we will use the output of the USDA NASS Cropland Data Layers ([gee](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL)) as training data.

## Configure the Environment

We begin by importing a number of useful libraries

In [32]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image
from functools import reduce
import h5py

Initialize the Earth Engine client.

In [33]:
ee.Initialize()

**Functions**

In [34]:
def display_image(image, region, Vizz = None):
    """
    Displays images in notebook
    """ 
    ## Visualization
    if Vizz:
        image = image.visualize(**Vizz)
        
    visual = Image(url=image.getThumbUrl({
                'region':region
                }))
    
    display(visual)

In [35]:
def CloudMaskS2(image):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
    parsed by Nick Clinton
    """
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # Both flags set to zero indicates clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
            qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [36]:
def CloudFreeCompositeS2(Collection_id, startDate, stopDate, geom):
    ## Define your collection
    collection = ee.ImageCollection(Collection_id)

    ## Filter 
    collection = collection.filterBounds(geom).filterDate(startDate,stopDate)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
            .map(CloudMaskS2)

    ## Composite
    composite = collection.median()
    
    return composite

## Sentinel 2  
### Sentinel-2 MSI: MultiSpectral Instrument, Level-1C ([gee](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2))
**Dataset Availability**: 2015-06-23T00:00:00 - Present

**Wavebands**

|Band 	|Use 		|Wavelength (nm) |Resolution (m)|
|-------|-----------|----------------|--------------|
|B1 	|Aerosols 	|443 	|60|
|B2 	|Blue 		|490 	|10|
|B3 	|Green 		|560 	|10|
|B4 	|Red 		|665 	|10|
|B6 	|Red Edge 2 |740 	|20|
|B8 	|NIR        |835 	|10|
|B8a 	|Red Edge 4 |865 	|20|
|B9 	|Water vapor|940 	|60|
|B10 	|Cirrus 	|1375 	|60|
|B11 	|SWIR 1 	|1610 	|20|
|B12 	|SWIR 2 	|2190 	|20|
|QA60   |ESA Cloud  | n/a   |60|

In [37]:
# GEE Image Collection ID
Collection_id = 'COPERNICUS/S2'
# Start and stop of time series
startDate = ee.Date('2016-01-01')
stopDate  = ee.Date('2016-12-31')
# Scale in meters
scale = 10

**Cloud Free Composite**

RGB

In [38]:
# Area of Interest (AoI)
geom = ee.Geometry.Point(-112.4007, 43.1805).buffer(12000)
region = geom.bounds().getInfo()['coordinates']
# Visualization parameters
vis = {'min':0,'max':0.3, 'bands':['B4','B3','B2']}
# Cloud Free Composite
image = CloudFreeCompositeS2(Collection_id, startDate, stopDate, geom)
# Display Composite
display_image(image, region, Vizz = vis)

NIR

In [39]:
# Visualization parameters
vis = {'min':0,'max':0.5, 'gamma':1.5, 'bands':['B8']}
# Display Composite
display_image(image, region, Vizz = vis)

NDVI = (RED-NIR)/(RED+NIR)

In [40]:
# Visualization parameters
palette = ['blue', 'white', 'green']
vis = {'min': -0.8, 'max': 0.8, 'bands':'nd', 'palette': palette}
# Calculate NDVI
image_ndvi = image.normalizedDifference(['B8','B4'])
# Display NDVI
display_image(image_ndvi, region, Vizz = vis)

NDWI = (GREEN-NIR)/(GREEN+NIR)

In [41]:
# Visualization parameters
palette = ['blue', 'white', 'green']
vis = {'min': -0.8, 'max': 0.8, 'bands':'nd', 'palette': palette}
# Calculate NDWI
image_ndwi = image.normalizedDifference(['B8','B3'])
# Display NDWI
display_image(image_ndwi, region, Vizz = vis)

## Cropland Data Layers
### USDA NASS Cropland Data Layers ([gee](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL))

**Dataset Availability**: January 1997 - Present

**Resolution**
30 meters

**Bands**

|Name 	    |Min|Max |Description 	|
|-----------|---|----|--------------|
|cropland 	|1 	|254 |Main crop-specific land cover classification.|
|cultivated |1 	|2   |Classification layer for identifying cultivated and non-cultivated land cover. Available from 2013 to 2017.|
|confidence |0 	|100 |Per-pixel predicted confidence of the given classification, with 0 being the least confident and 100 the most confident.|

In [29]:
# GEE Image Collection ID
Collection_id = 'USDA/NASS/CDL'

Ground truth land cover classification

In [30]:
dataset = ee.ImageCollection(Collection_id)\
    .filterBounds(geom)\
    .filterDate(startDate,stopDate)

# First image
image = ee.Image(dataset.first())

# Choose the scale
image =  image.reproject(crs='EPSG:4326', scale=scale)

In [31]:
vis = {'min':1,'max':254, 'bands':'cropland'}
display_image(image, region, Vizz = vis)

## Download datasets
We download and stack datasets from two different Areas of Interest (AOIs)

In [ ]:
# Central position of (AOIs)
points = [[-120.7224, 37.3872], [-112.6799, 42.9816]]

In [ ]:
from ee_datasets import ee_datasets

for n, point in enumerate(points):
    sentinel = ee_datasets(point = point, buffer = 10000 , startDate = startDate, stopDate = stopDate, scale = scale, collection = 'Sentinel2')
    cropland = ee_datasets(point = point, buffer = 10000 , startDate = startDate, stopDate = stopDate, scale = scale, collection = 'CroplandDataLayers')
    dataset_x = sentinel.read_datasets()
    dataset_y = cropland.read_datasets()
    if n == 0:
        data_x = dataset_x
        data_y = dataset_y
    else:
        szy1, szx1 = data_x.shape[:2]
        szy2, szx2 = dataset_x.shape[:2]
        if szy1 != szy2 or szx1 != szx2:
            szy = min(szy1, szy2)
            szx = min(szx1, szx2)
            
            data_x = np.stack((data_x[:szy,:szx,:], dataset_x[:szy,:szx,:]), axis=0)
            data_y = np.stack((data_y[:szy,:szx,:], dataset_y[:szy,:szx,:]), axis=0)
        else:
            data_x = np.stack((data_x, dataset_x), axis=0)
            data_y = np.stack((data_y, dataset_y), axis=0)

**Display channels**

We display the input and output datasets

In [ ]:
def display_channels(data, nChannels, titles = False):
    if nChannels == 1:
        plt.figure(figsize=(5,5))
        plt.imshow(data[:,:,0])
        if titles:
            plt.title(titles[0])
    else:
        fig, axs = plt.subplots(nrows=1, ncols=nChannels, figsize=(5*nChannels,5))
        for i in range(nChannels):
            ax = axs[i]
            ax.imshow(data[:,:,i])
            if titles:
                ax.set_title(titles[i])

Sentinel 2 composite for the for the fist AOI

In [ ]:
display_channels(data_x[0,:,:,:], data_x.shape[3], titles=['Blue', 'Green', 'Red', 'NIR', 'NDVI', 'NDWI'])

Ground truth land cover classification for the for the fist AOI

In [ ]:
display_channels(data_y[0,:,:,:], data_y.shape[3], titles=['Cropland'])

Sentinel 2 composite for the for the second AOI

In [ ]:
display_channels(data_x[1,:,:,:], data_x.shape[3], titles=['Blue', 'Green', 'Red', 'NIR', 'NDVI', 'NDWI'])

Ground truth land cover classification for the for the second AOI

In [ ]:
display_channels(data_y[1,:,:,:], data_y.shape[3], titles=['Cropland'])

## Preprocess class labels

Each class in encoded as a value in the range between 0 to 254. For training a Neural Network in Keras we have to convert the 1-dimensional class arrays to N classes-dimensional matrices. To simplify the problem here we regroup all the classes into 4 categories, namely, land, water, urban, and cropland areas.

In [ ]:
# Area of Interest (AoI)
geom = ee.Geometry.Point(points[0]).buffer(10000)
# Start and stop of time series
startDate = ee.Date('2016')
stopDate  = ee.Date('2017')
# Read the ImageCollection
dataset = ee.ImageCollection('USDA/NASS/CDL')\
    .filterBounds(geom)\
    .filterDate(startDate,stopDate)
# Get the cropland class values and names
cropland_info = pd.DataFrame({'cropland_class_values':dataset.getInfo().get('features')[0].get('properties').get('cropland_class_values'),
                              'cropland_class_palette':dataset.getInfo().get('features')[0].get('properties').get('cropland_class_palette'),
                              'cropland_class_names':dataset.getInfo().get('features')[0].get('properties').get('cropland_class_names')
                             })
cropland_info.head()

The number of unique classes in this are is equal to:

In [ ]:
len(np.unique(data_y[:,:,:,0]))

and the number of pixels by class

In [ ]:
value, count = np.unique(data_y[0,:,:,0], return_counts=True)
df = pd.DataFrame({'cropland_class_values': value, 'cropland_class_counts': count})
df.sort_values(by='cropland_class_counts', ascending=False, inplace=True)
df = pd.merge(df, cropland_info, how='left', on=['cropland_class_values'])
df.head()

Create new classes:
- Land
- Water
- Urban
- Croplands

In [ ]:
def replace_values(array, class_labels, new_label):
    array_new = np.copy(array)
    for i in range(len(class_labels)):
        array_new[np.where(array == class_labels[i])] = new_label
        
    return array_new

In [ ]:
# New classes
land = ['Shrubland', 'Barren', 'Grassland/Pasture', 'Deciduous Forest', 'Evergreen Forest', 'Mixed Forest', 'Wetlands', 'Woody Wetlands', 'Herbaceous Wetlands']
water = ['Water', 'Open Water', 'Aquaculture']
urban = ['Developed', 'Developed/Open Space', 'Developed/High Intensity', 'Developed/Low Intensity', 'Developed/Med Intensity']

class_labels_0 = np.array(cropland_info[cropland_info['cropland_class_names'].isin(land)]['cropland_class_values'])
class_labels_1 = np.array(cropland_info[cropland_info['cropland_class_names'].isin(water)]['cropland_class_values'])
class_labels_2 = np.array(cropland_info[cropland_info['cropland_class_names'].isin(urban)]['cropland_class_values'])
class_labels_3 = np.array(cropland_info[(~cropland_info['cropland_class_names'].isin(land)) & 
                                        (~cropland_info['cropland_class_names'].isin(water)) & 
                                        (~cropland_info['cropland_class_names'].isin(urban))]['cropland_class_values'])

# We replace the class labels
new_data_y = np.copy(data_y[:,:,:,0])
new_data_y = replace_values(new_data_y, class_labels_3, 3.)
new_data_y = replace_values(new_data_y, class_labels_2, 2.)
new_data_y = replace_values(new_data_y, class_labels_1, 1.)
new_data_y = replace_values(new_data_y, class_labels_0, 0.)

# Convert 1-dimensional class arrays to 4-dimensional class matrices
from keras.utils import np_utils
new_data_y = np_utils.to_categorical(new_data_y, 4)
data_y = new_data_y

New classification for the for the first AOI

In [ ]:
display_channels(data_y[0,:,:,:], data_y.shape[3], titles=['Land', 'Water', 'Urban', 'Cropland'])

New classification for the for the second AOI

In [ ]:
display_channels(data_y[1,:,:,:], data_y.shape[3], titles=['Land', 'Water', 'Urban', 'Cropland'])

**Select input channels**

We will only use NIR, NDVI, and NDWI as an input channels

In [ ]:
data_x = data_x[:,:,:,3:]

## Preprocess datasets for training a Fully Connected Network (FCN)

**Normalize data**

In [ ]:
def normalize_data(data):
    size = data.shape
    for i in range(size[-1]):
        mx = data[:,:,:,i].max()
        mn = data[:,:,:,i].min()
        
        data[:,:,:,i] = (data[:,:,:,i]-mn)/(mx-mn)
    return data

In [ ]:
data_x = normalize_data(data_x)

**Resize the images**

Current shape

In [ ]:
data_x.shape

In [ ]:
data_y.shape

In [ ]:
def reshape_data(data):
    size = data.shape
    new_size = []
    new_size.append(reduce(lambda x, y: x*y, size[:-1]))
    new_size.append(size[-1])
    new_size = tuple(new_size)
    return size, new_size

In [ ]:
size_x, new_size_x = reshape_data(data_x)
size_y, new_size_y = reshape_data(data_y)

data_x_new = data_x.reshape(new_size_x)
data_y_new = data_y.reshape(new_size_y)

New shape

In [ ]:
data_x_new.shape

In [ ]:
data_y_new.shape

**Randomize the datasets**

In [ ]:
def randomize_datasets(data_x, data_y):
    t=data_x.shape[0]
    arr_t = np.arange(t)
    np.random.shuffle(arr_t)
    data_x = data_x[arr_t,:]
    data_y = data_y[arr_t,:]
    
    return data_x, data_y

In [ ]:
x_randm, y_randm = randomize_datasets(data_x_new, data_y_new)

**Training and validation sets**

In [ ]:
def train_validation_split(x, y, val_size=20):
    t=x.shape[0]
    size = int(t*((100-val_size)/100))
    
    xt = x[:size,:]
    xv = x[size:,:]
    yt = y[:size,:]
    yv = y[size:,:]
    
    return xt, xv, yt, yv

In [ ]:
x_train, x_validation, y_train, y_validation = train_validation_split(x_randm, y_randm)

## Define the Keras model

Here we define a neural network with two hidden layers with `relu` nonlinearities.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

batch_size = 32
num_bands = 3
num_classes = 4
epochs = 1

model = Sequential()

model.add(Dense(512, activation = 'relu', input_shape=(num_bands,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation = 'softmax'))

model.summary()

**Compile the model**

In [ ]:
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

**Train the Neural Network**

In [ ]:
from keras.callbacks import Callback, ModelCheckpoint

In [ ]:
# To saves the model weights after each epoch if the validation loss decreased
checkpointer = ModelCheckpoint(filepath="{0}_weights.hdf5".format('FCN'), verbose=1, save_best_only=True)

In [ ]:
model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_validation, y_validation), callbacks=[checkpointer])

**Evaluate model**

In [ ]:
score = model.evaluate(x_validation, y_validation, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

**Read the weights**

In [ ]:
batch_size = 32
num_bands = 3
num_classes = 4
epochs = 1

model = Sequential()
model = Sequential()

model.add(Dense(512, activation = 'relu', input_shape=(num_bands,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation = 'softmax'))

model.load_weights("{0}_weights.hdf5".format('FCN'))

## Predict

First we test the FCN in a AOI that is close to one of the training AOI.

In [ ]:
# Central position of (AOIs)
point = [-112.4336, 43.1682]

In [ ]:
sentinel = ee_datasets(point = point, buffer = 10000 , startDate = startDate, stopDate = stopDate, scale = scale, collection = 'Sentinel2')
dataset_x = sentinel.read_datasets()
cropland = ee_datasets(point = point, buffer = 10000 , startDate = startDate, stopDate = stopDate, scale = scale, collection = 'CroplandDataLayers')
dataset_y = cropland.read_datasets()

We select the NIR, NDVI, NDWI channels

In [ ]:
data_x = dataset_x[:,:,3:]
data_y = dataset_y

Sentinel 2 composite

In [ ]:
display_channels(data_x, data_x.shape[2], titles=['NIR', 'NDVI', 'NDWI'])

Ground truth land cover classification

In [ ]:
display_channels(data_y, data_y.shape[2], titles=['Cropland'])

**Preprocess class labels**

In [ ]:
# We replace the class labels
new_data_y = np.copy(data_y[:,:,0])
new_data_y = replace_values(new_data_y, class_labels_3, 3.)
new_data_y = replace_values(new_data_y, class_labels_2, 2.)
new_data_y = replace_values(new_data_y, class_labels_1, 1.)
new_data_y = replace_values(new_data_y, class_labels_0, 0.)

# Convert 1-dimensional class arrays to 4-dimensional class matrices
from keras.utils import np_utils
new_data_y = np_utils.to_categorical(new_data_y, 4)

Output classes

In [ ]:
display_channels(new_data_y, new_data_y.shape[2], titles=['Land', 'Water', 'Urban', 'Cropland'])

**Preprocess input dataset**

In [ ]:
# Normalize
def normalize_data(data):
    size = data.shape
    for i in range(size[-1]):
        mx = data[:,:,i].max()
        mn = data[:,:,i].min()
        
        data[:,:,i] = (data[:,:,i]-mn)/(mx-mn)
    return data
data_x_norm = normalize_data(data_x)

In [ ]:
# Normalize
data_x_norm = normalize_data(data_x)
# Resize
size_x, new_size_x = reshape_data(data_x_norm)

x_input = data_x_norm.reshape(new_size_x)

**Compute the prediction**

In [ ]:
y_output = model.predict(x_input, batch_size=batch_size, verbose=0)

**Resize the output**

In [ ]:
data_y_output = y_output.reshape((size_x[0], size_x[1],4))

**Display the output**

In [ ]:
display_channels(data_y_output, data_y_output.shape[2], titles=['Land', 'Water', 'Urbar', 'Cropland'])

We binarize the output taking the highest pixel value

In [ ]:
# Binarize the output
def max_pixels(x):
    x_new = x*0
    max_val = np.amax(x, axis=2)
    size = x.shape
    for i in range(size[-1]):
        ima = x[:,:,i]*0
        ima[np.where(x[:,:,i] == max_val)] = 1
        x_new[:,:,i]= ima

    return x_new

In [ ]:
data_y_output_max = max_pixels(data_y_output)

In [ ]:
display_channels(data_y_output_max, data_y_output.shape[2], titles=['Land', 'Water', 'Urban', 'Cropland'])

If we compare the ground truth with the prediction we find that the FCN performes quite well for the land, water, and cropland areas but it fails in detecting urban areas.

First we test the FCN in a AOI that is very different from those used for training.

In [ ]:
point = [19.7368, -17.9489]

In [ ]:
from Landsat_dataset import landsat_datasets
landsat = landsat_datasets(point = point, buffer = 12000 , startDate = '2014', stopDate = '2017', scale = 30)
dataset_x = landsat.read_datasets()

We select the NIR and NDVI channels

In [ ]:
dataset_x = dataset_x[:,:,3:]

Lansat 7 composite

In [ ]:
display_channels(dataset_x, dataset_x.shape[2], titles=['NIR', 'NDVI'])

**Preprocess input dataset**

In [ ]:
# Normalize
dataset_x_norm = normalize_data(dataset_x)
# Resize 
size_x, new_size_x = reshape_data(dataset_x_norm)

x_input = dataset_x_norm.reshape(new_size_x)

**Compute the prediction**

In [ ]:
y_output = model.predict(x_input, batch_size=batch_size, verbose=0)

**Resize the output**

In [ ]:
data_y_output = y_output.reshape((size_x[0], size_x[1],4))

**Display the output**

In [ ]:
display_channels(data_y_output, data_y_output.shape[2], titles=['Land', 'Water', 'Urbar', 'Cropland'])

In [ ]:
data_y_output_max = max_pixels(data_y_output)
display_channels(data_y_output_max, data_y_output.shape[2], titles=['Land', 'Water', 'Urbar', 'Cropland'])